In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
!pip install fastai2
from IPython import display
display.clear_output()
from fastai2.basics import *
from fastai2.tabular.all import *

import pandas as pd, numpy as np, matplotlib.pyplot as plt
df = pd.read_csv('doctor_train.csv')
test_df = pd.read_csv('doctor_test.csv')

In [0]:
num_col = ['age','Money','Doctor_visits','last_visit','cured_in']
for i in num_col:
  df[i+'_c'] = df[i]
for i in num_col:
  test_df[i+'_c'] = test_df[i]

In [43]:
procs = [FillMissing, Categorify, Normalize]
cat_col = ['Profession','Status','edu','Irregular','residence','prev_diagnosed',
           'communication','side_effects','age_c',
           'Money_c', 'Doctor_visits_c', 'last_visit_c', 'cured_in_c']
dep_var = ['Y']

splits = RandomSplitter()(range_of(df))
to = TabularPandas(df, procs, cat_names=cat_col, cont_names=num_col, y_names=dep_var, splits=splits)
dls = to.dataloaders(bs=2048)

IndexError: ignored

In [0]:
class myLabelSmoothingCrossEntropyFlat(BaseLoss):
    "Same as `nn.CrossEntropyLoss`, but flattens input and target."
    y_int = True
    def __init__(self, *args, axis=-1, **kwargs): super().__init__(LabelSmoothingCrossEntropy, *args, axis=axis, **kwargs)
    def decodes(self, x):    return x.argmax(dim=self.axis)
    def activation(self, x): return F.softmax(x, dim=self.axis)
    
class TrackHighestEpoch(TrackerCallback):
    "A `TrackerCallback` that terminates training when monitored quantity stops improving."
    def __init__(self, monitor='accuracy', comp=None, min_delta=0., patience=1):
        super().__init__(monitor=monitor, comp=comp, min_delta=min_delta)
        self.patience = patience

    def begin_fit(self): self.highest_epoch = 0; super().begin_fit()
    def after_epoch(self):
        "Compare the value monitored to its best score and maybe stop training."
        super().after_epoch()
        if self.new_best: 
          self.highest_epoch = self.epoch
    def after_fit(self):
        "Print highest value"
        print(f'Highest accuracy of: {self.best} at epoch {self.highest_epoch+1}')

In [40]:
learn = tabular_learner(dls, layers=[1000,500,100,50,50],
                        ps=[0.01,0.01,0.01,0.01,0.02], metrics=accuracy,
                        n_out=2, cbs=TrackHighestEpoch)
learn.fit_one_cycle(10, 1e-1)

epoch,train_loss,valid_loss,accuracy,time
0,0.401640,0.412505,0.878347,00:06
1,0.344468,0.453168,0.882603,00:06
2,0.307317,0.465268,0.888278,00:06
3,0.277525,0.525425,0.865402,00:06
4,0.251393,0.485537,0.878702,00:06
5,0.228801,0.515889,0.863451,00:06
6,0.208499,0.614147,0.866820,00:06
7,0.188022,0.677063,0.867175,00:06
8,0.168767,0.751718,0.865934,00:06
9,0.152143,0.759167,0.864160,00:06


Highest accuracy of: 0.888278067111969 at epoch 3


In [41]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)
pred_prob, pred_class = preds.max(1)

le_y = le.inverse_transform(pred_class)

output = pd.DataFrame({'id': test_df.ID,
                       'Y': le_y})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')

NameError: ignored